# Import Packages

In [60]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

# Helper Functions

In [61]:
    def grayscale(img):
        """Applies the Grayscale transform
         This will return an image with only one color channel
        but NOTE: to see the returned image as grayscale
        (assuming your grayscaled image is called 'gray')
        you should call plt.imshow(gray, cmap='gray')"""
        return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    def gaussian_blur(img, kernel_size):
        """Applies a Gaussian Noise kernel"""
        return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

    def canny(img, low_threshold, high_threshold):
        """Applies the Canny transform"""
        return cv2.Canny(img, low_threshold, high_threshold)

    def region_of_interest(img, vertices):
        """
         Applies an image mask.
    
        Only keeps the region of the image defined by the polygon
        formed from `vertices`. The rest of the image is set to black.
        """
        #defining a blank mask to start with
        mask = np.zeros_like(img)   
    
        #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
        if len(img.shape) > 2:
            channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
            ignore_mask_color = (255,) * channel_count
        else:
            ignore_mask_color = 255
        
        #filling pixels inside the polygon defined by "vertices" with the fill color    
        cv2.fillPoly(mask, vertices, ignore_mask_color)
    
        #returning the image only where mask pixels are nonzero
        masked_image = cv2.bitwise_and(img, mask)
        return masked_image

    def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
        """
        `img` should be the output of a Canny transform.
        
        Returns an image with hough lines drawn.
        """
        lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    
        return lines  
    def linear_reg(Hough_array, linetype, line_image, out_type):
        
        '''Applies linear regression for specific slope ranges linetype='r' 
            for right side line and 'l' for left side lines. '''
        
        temp_list=[]
        for line in Hough_array:
            for x1,y1,x2,y2 in line:
                slope= (y2-y1)/(x2-x1)
                if linetype=='l' and slope<-0.5 and slope>-0.8:
                    temp_list.append([x1,x2,y1,y2])
                    #this condition will execute if we just want to see output like raw_video sampl
                    if out_type =='raw':
                        cv2.line(line_image, (x1,y1), (x2,y2), [255, 0, 100], 8)
                elif linetype=='r' and slope>0.5 and slope<0.8:
                    temp_list.append([x1,x2,y1,y2])
                    if out_type =='raw':
                        cv2.line(line_image, (x1,y1), (x2,y2), [255, 0,100], 8)
        temp = np.array(temp_list)
        X=np.reshape(temp[:,[0,1]], (1,len(temp)*2))[0]
        A = np.vstack([X,np.ones(len(X))]).T
        Y = np.reshape(temp[:, [2, 3]], (1, len(temp) * 2))[0]
        m, c = np.linalg.lstsq(A, Y)[0]
        X= np.array(X)
        Y = np.array(X * m + c)
        minY= Y.min()
        maxY= Y.max()
        return minY, maxY,m,c,line_image

    

# Main Function

In [62]:
def process_image(image):
    
     #Converting the image to grayscale
    gray = grayscale(image)
    
    
    #Applying gaussian blur to the grayscale image
    kernel_size = 7
    blur_gray = gaussian_blur(gray, kernel_size)
    
    
    # Define our parameters for Canny and run it
    low_threshold =50
    high_threshold = 200
    edges = canny(blur_gray, low_threshold, high_threshold)

    # Next we'll create a masked edges image using cv2.fillPoly()
    
    vertices_left = np.array([[(40,500),(390, 350), (500, 350), (500,500)]], dtype=np.int32)
    vertices_right = np.array([[(940,500),(650, 350), (500, 350), (500,500)]], dtype=np.int32)
   
    masked_edges_left = region_of_interest(edges, vertices_left)
    masked_edges_right = region_of_interest(edges, vertices_right)
    
        # Make a blank the same size as our image to draw on
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 10    # minimum number of votes (intersections in Hough grid cell)
    min_line_length =10 #minimum number of pixels making up a line
    max_line_gap = 15  # maximum gap in pixels between connectable line segments 15

    line_image = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
  
    
    # Apply Hough's transform for left side and right side lines
    hough_lines_left = hough_lines(masked_edges_left, rho, theta, threshold, min_line_length, max_line_gap)
    hough_lines_right = hough_lines(masked_edges_right, rho, theta, threshold, min_line_length, max_line_gap)
    
    #Run linear regression for selected slope range 
    minY_right, maxY_right, m_right, c_right, line_image = linear_reg(hough_lines_right, 'r', line_image, 0)
    minY_left, maxY_left, m_left, c_left, line_image = linear_reg(hough_lines_left, 'l',line_image, 0)
    
    # Determine the upper and lower points to draw a line in the image
    min_y = np.min([minY_left, minY_right])
    upper_right_pt = np.array([(min_y - c_right) / m_right, min_y], dtype=int)
    upper_left_pt = np.array([(min_y - c_left) / m_left, min_y], dtype=int)
    max_y = np.max([maxY_left,maxY_right])
    lower_left_pt = np.array([(max_y - c_left) / m_left, max_y], dtype=int)
    lower_right_pt = np.array([(max_y - c_right) / m_right, max_y], dtype=int)


    # Draw the lines.
    cv2.line(line_image, (lower_left_pt[0], lower_left_pt[1]), (upper_left_pt[0], upper_left_pt[1]), [255, 0, 0], 10)
    cv2.line(line_image, (lower_right_pt[0], lower_right_pt[1]), (upper_right_pt[0], upper_right_pt[1]), [255, 0, 0], 10)
    lines_edges = cv2.addWeighted(image, 0.8, line_image, 10, 0)
    return lines_edges
    

# Video Inputs

In [63]:
#Processing white video input

white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip= clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)


[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|█████████▉| 221/222 [00:02<00:00, 81.53it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 2.99 s, sys: 874 ms, total: 3.87 s
Wall time: 3.05 s


In [64]:
#Processing yellow video input

yellow_output ='yellow.mp4'
clip1 = VideoFileClip("solidYellowLeft.mp4")
yellow_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|█████████▉| 681/682 [00:08<00:00, 80.72it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

CPU times: user 9.19 s, sys: 2.5 s, total: 11.7 s
Wall time: 8.77 s


# Outputs annotated line

In [65]:
#View the processed video of white_output

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [66]:
#View the processed video of yellow_output

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

# To view the raw lines output 

In [70]:
def process_image_raw(image):
    
     #Converting the image to grayscale
    gray = grayscale(image)
    
    
    #Applying gaussian blur to the grayscale image
    kernel_size = 7
    blur_gray = gaussian_blur(gray, kernel_size)
    
    
    # Define our parameters for Canny and run it
    low_threshold =50
    high_threshold = 200
    edges = canny(blur_gray, low_threshold, high_threshold)

    # Next we'll create a masked edges image using cv2.fillPoly()
    
    vertices_left = np.array([[(40,500),(390, 350), (500, 350), (500,500)]], dtype=np.int32)
    vertices_right = np.array([[(940,500),(650, 350), (500, 350), (500,500)]], dtype=np.int32)
   
    masked_edges_left = region_of_interest(edges, vertices_left)
    masked_edges_right = region_of_interest(edges, vertices_right)
    
        # Make a blank the same size as our image to draw on
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 10    # minimum number of votes (intersections in Hough grid cell)
    min_line_length =10 #minimum number of pixels making up a line
    max_line_gap = 15  # maximum gap in pixels between connectable line segments 15

    line_image = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
  
    
    # Apply Hough's transform for left side and right side lines
    hough_lines_left = hough_lines(masked_edges_left, rho, theta, threshold, min_line_length, max_line_gap)
    hough_lines_right = hough_lines(masked_edges_right, rho, theta, threshold, min_line_length, max_line_gap)
    
    #Run linear regression for selected slope range 
    minY_right, maxY_right, m_right, c_right, line_image = linear_reg(hough_lines_right, 'r', line_image, 'raw')
    minY_left, maxY_left, m_left, c_left, line_image = linear_reg(hough_lines_left, 'l',line_image, 'raw')
    
    # Determine the upper and lower points to draw a line in the image
    min_y = np.min([minY_left, minY_right])
    upper_right_pt = np.array([(min_y - c_right) / m_right, min_y], dtype=int)
    upper_left_pt = np.array([(min_y - c_left) / m_left, min_y], dtype=int)
    max_y = np.max([maxY_left,maxY_right])
    lower_left_pt = np.array([(max_y - c_left) / m_left, max_y], dtype=int)
    lower_right_pt = np.array([(max_y - c_right) / m_right, max_y], dtype=int)


    # Draw the lines. the below lines are commented in the case of viewing the raw lines in the video output
    #cv2.line(line_image, (lower_left_pt[0], lower_left_pt[1]), (upper_left_pt[0], upper_left_pt[1]), [255, 0, 0], 10)
    #cv2.line(line_image, (lower_right_pt[0], lower_right_pt[1]), (upper_right_pt[0], upper_right_pt[1]), [255, 0, 0], 10)
    lines_edges = cv2.addWeighted(image, 0.8, line_image, 10, 0)
    return lines_edges
    

# Raw lines output for SolidWhiteRight

In [71]:
white_output = 'white_raw.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip= clip1.fl_image(process_image_raw) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

#View the processed video of white_output

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))


[MoviePy] >>>> Building video white_raw.mp4
[MoviePy] Writing video white_raw.mp4


100%|█████████▉| 221/222 [00:02<00:00, 81.01it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white_raw.mp4 

CPU times: user 3.01 s, sys: 871 ms, total: 3.88 s
Wall time: 3.07 s


# Raw lines output for SolidYellowLeft


In [72]:
#Processing yellow video input

yellow_output ='yellow_raw.mp4'
clip1 = VideoFileClip("solidYellowLeft.mp4")
yellow_clip = clip1.fl_image(process_image_raw) #NOTE: this function expects color images!!
%time yellow_clip.write_videofile(yellow_output, audio=False)

#View the processed video of yellow_output

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

[MoviePy] >>>> Building video yellow_raw.mp4
[MoviePy] Writing video yellow_raw.mp4


100%|█████████▉| 681/682 [00:08<00:00, 77.79it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow_raw.mp4 

CPU times: user 9.46 s, sys: 2.58 s, total: 12 s
Wall time: 9.12 s
